In [1]:
from __future__ import absolute_import, division, print_function



import tensorflow as tf



# Set Eager API

tf.enable_eager_execution()

tfe = tf.contrib.eager



# Import MNIST data

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)



# Parameters

learning_rate = 0.1

batch_size = 128

num_steps = 1000

display_step = 100



dataset = tf.data.Dataset.from_tensor_slices(

    (mnist.train.images, mnist.train.labels))

dataset = dataset.repeat().batch(batch_size).prefetch(batch_size)

dataset_iter = tfe.Iterator(dataset)



# Variables

W = tfe.Variable(tf.zeros([784, 10]), name='weights')

b = tfe.Variable(tf.zeros([10]), name='bias')





# Logistic regression (Wx + b)

def logistic_regression(inputs):

    return tf.matmul(inputs, W) + b





# Cross-Entropy loss function

def loss_fn(inference_fn, inputs, labels):

    # Using sparse_softmax cross entropy

    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(

        logits=inference_fn(inputs), labels=labels))





# Calculate accuracy

def accuracy_fn(inference_fn, inputs, labels):

    prediction = tf.nn.softmax(inference_fn(inputs))

    correct_pred = tf.equal(tf.argmax(prediction, 1), labels)

    return tf.reduce_mean(tf.cast(correct_pred, tf.float32))





# SGD Optimizer

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

# Compute gradients

grad = tfe.implicit_gradients(loss_fn)



# Training

average_loss = 0.

average_acc = 0.

for step in range(num_steps):



    # Iterate through the dataset

    d = dataset_iter.next()



    # Images

    x_batch = d[0]

    # Labels

    y_batch = tf.cast(d[1], dtype=tf.int64)



    # Compute the batch loss

    batch_loss = loss_fn(logistic_regression, x_batch, y_batch)

    average_loss += batch_loss

    # Compute the batch accuracy

    batch_accuracy = accuracy_fn(logistic_regression, x_batch, y_batch)

    average_acc += batch_accuracy



    if step == 0:

        # Display the initial cost, before optimizing

        print("Initial loss= {:.9f}".format(average_loss))



    # Update the variables following gradients info

    optimizer.apply_gradients(grad(logistic_regression, x_batch, y_batch))



    # Display info

    if (step + 1) % display_step == 0 or step == 0:

        if step > 0:

            average_loss /= display_step

            average_acc /= display_step

        print("Step:", '%04d' % (step + 1), " loss=",

              "{:.9f}".format(average_loss), " accuracy=",

              "{:.4f}".format(average_acc))

        average_loss = 0.

        average_acc = 0.



# Evaluate model on the test image set

testX = mnist.test.images

testY = mnist.test.labels



test_acc = accuracy_fn(logistic_regression, testX, testY)

print("Testset Accuracy: {:.4f}".format(test_acc))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfu